In [2]:
import keras
from IPython.display import Image

In [3]:
import os
import random
import re
from PIL import Image

# Rozdelenie snimkov na train, val a test

In [8]:
DATA_PATH = 'D:\Dataset\\'
FRAME_PATH = DATA_PATH +'Snimky\\'
MASK_PATH = DATA_PATH +'Mask\\'


In [9]:
# Create folders to hold images and masks

folders = ['train_frames\\train', 'train_masks\\train', 'val_frames\\val', 'val_masks\\val', 'test_frames\\test', 'test_masks\\test']


for folder in folders:
    os.makedirs(DATA_PATH + folder)

In [10]:
all_frames = os.listdir(FRAME_PATH)
all_masks = os.listdir(MASK_PATH)

In [11]:
all_frames.sort(key=lambda var:[int(x) if x.isdigit() else x 
                                for x in re.findall(r'[^0-9]|[0-9]+', var)])
all_masks.sort(key=lambda var:[int(x) if x.isdigit() else x 
                               for x in re.findall(r'[^0-9]|[0-9]+', var)])

In [217]:
#  NEROBIT TOTO LEBO TO PREMIESA A POTOM TO NENAMAPUJE PRESNE TAK AKO CHCEME
# random.seed(21)
# random.shuffle(all_frames)

In [12]:
# Generate train, val, and test sets for frames

train_split = int(0.7*len(all_frames))
val_split = int(0.9 * len(all_frames))

train_frames = all_frames[:train_split]
val_frames = all_frames[train_split:val_split]
test_frames = all_frames[val_split:]

In [13]:
# Generate corresponding mask lists for masks

train_masks = [f for f in all_masks if f in train_frames]
val_masks = [f for f in all_masks if f in val_frames]
test_masks = [f for f in all_masks if f in test_frames]

In [14]:
def add_frames(dir_name, image):
  
  img = Image.open(FRAME_PATH+image)
  img.save(DATA_PATH+'/{}'.format(dir_name)+'\\'+image)
  
  
  
def add_masks(dir_name, image):
  
  img = Image.open(MASK_PATH+image)
  img.save(DATA_PATH+'/{}'.format(dir_name)+'\\'+image)

In [15]:
frame_folders = [(train_frames, 'train_frames\\train'), (val_frames, 'val_frames\\val'), 
                 (test_frames, 'test_frames\\test')]

mask_folders = [(train_masks, 'train_masks\\train'), (val_masks, 'val_masks\\val'), 
                (test_masks, 'test_masks\\test')]


In [16]:
for folder in frame_folders:
  
  array = folder[0]
  name = [folder[1]] * len(array)

  list(map(add_frames, name, array))

In [17]:
for folder in mask_folders:
  
  array = folder[0]
  name = [folder[1]] * len(array)
  
  list(map(add_masks, name, array))

# Vytvorenie generatoru

## Potrebna organizacia priecinkov

![dir_organisation](priecinky.jpg)

## definovanie generatora

In [18]:
from keras.preprocessing.image import ImageDataGenerator

In [19]:
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()

In [37]:
train_image_generator = train_datagen.flow_from_directory(
'D:/Dataset/train_frames',
    target_size = (320,320),
    class_mode= None,
    batch_size = 2)

train_mask_generator = train_datagen.flow_from_directory(
'D:/Dataset/train_masks',
    target_size = (320,320),
    class_mode= None,
    color_mode = 'grayscale',
batch_size = 2)


val_image_generator = val_datagen.flow_from_directory(
'D:/Dataset/val_frames',
class_mode= None,
    color_mode = 'grayscale',
batch_size = 2)


val_mask_generator = val_datagen.flow_from_directory(
'D:/Dataset/val_masks',
    target_size = (320,320),
    class_mode= None,
    color_mode = 'grayscale',
batch_size = 2)



train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)

Found 11 images belonging to 1 classes.
Found 11 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.


# Skuska generatoru

**Nacitanie modelu**

In [22]:
from src.Model import get_unet

In [39]:
input_img = keras.layers.Input(shape=(320,320,3))
model = get_unet(input_img)

In [25]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 320, 320, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 320, 320, 16) 64          conv2d_2[0][0]                   
__________________________________________________________________________________________________
activation_2 (Activation)       (None, 320, 320, 16) 0           batch_normalization_2[0][0]      
____________________________________________________________________________________________

**Trenovanie**

In [40]:
optimizer = keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['accuracy'],
              #sample_weight_mode='temporal'
              )

In [41]:
model.fit_generator(generator=train_generator,epochs=10,steps_per_epoch=5)

Epoch 1/10
5/5 [==============================] - 12s 2s/step - loss: 13145.4072 - accuracy: 0.2086
Epoch 2/10
5/5 [==============================] - 6s 1s/step - loss: 12996.4417 - accuracy: 0.3741
Epoch 3/10
5/5 [==============================] - 6s 1s/step - loss: 13116.7906 - accuracy: 0.5887
Epoch 4/10
5/5 [==============================] - 6s 1s/step - loss: 13125.3921 - accuracy: 0.6245
Epoch 5/10
5/5 [==============================] - 6s 1s/step - loss: 13072.0979 - accuracy: 0.6083
Epoch 6/10
5/5 [==============================] - 6s 1s/step - loss: 13108.5662 - accuracy: 0.5868
Epoch 7/10
5/5 [==============================] - 7s 1s/step - loss: 13110.7281 - accuracy: 0.6145
Epoch 8/10
5/5 [==============================] - 6s 1s/step - loss: 13035.2666 - accuracy: 0.6273
Epoch 9/10
5/5 [==============================] - 6s 1s/step - loss: 13107.3308 - accuracy: 0.6425
Epoch 10/10
5/5 [==============================] - 6s 1s/step - loss: 13060.9034 - accuracy: 0.6468


# ulozenie modelu

In [ ]:
model.save('Model.h5')